# Basic Usage of EPyT-Flow

[EPyT-Flow](https://github.com/WaterFutures/EPyT-Flow) is a Python package building on top of [EPyT](https://github.com/OpenWaterAnalytics/EPyT) 
for providing easy access to water distribution network simulations.
It aims to provide a high-level interface for the easy generation of hydraulic and water quality scenario data.
However, it also provides access to low-level functions by [EPANET](https://github.com/USEPA/EPANET2.2) 
and [EPANET-MSX](https://github.com/USEPA/EPANETMSX/).

EPyT-Flow provides easy access to popular benchmark data sets for event detection and localization.
Furthermore, it also provides an environment for developing and testing control algorithms.

Unique features of EPyT-Flow that make it superior to other (Python) toolboxes are the following:

- High-performance hydraulic and (advanced) water quality simulation
- High- and low-level interface
- Object-orientated design that is easy to extend and customize
- Sensor configurations
- Wide variety of pre-defined events (e.g. leakages, sensor faults, actuator events, cyber-attacks, etc.)
- Wide variety of pre-defined types of uncertainties (e.g. model uncertainties)
- Step-wise simulation and environment for training and evaluating control strategies
- Serialization module for easy exchange of data and (scenario) configurations
- REST API to make EPyT-Flow accessible in other applications
- Access to many WDNs and popular benchmarks (incl. their evaluation)

### Outline

This notebook demonstrates the basic usage of EPyT-Flow:

1. Loading an existing (benchmark) scenario.
2. Working with the WDN topology:
    - Visualizing the network topology
    - Retrieving information about the network and solving common problems such as finding the shortest path between two nodes
4. Running a complete scenario simulation by:
    - Specifying general parameters such as the simulation duration and the hydraulic units.
    - Specifying a sensor placement
5. Post-processing the simulation results
    - Retrieving the final sensor readings
    - Plotting time series data

More details can be found in the [documentation](https://epyt-flow.readthedocs.io/en/stable) of EPyT-Flow.

[EPyT-Flow](https://github.com/WaterFutures/EPyT-Flow) is available on [PyPI](https://pypi.org/project/epyt-flow/) and can be installed via `pip install epyt-flow`:

In [ ]:
%pip install epyt-flow

In [ ]:
from epyt.epanet import ToolkitConstants
from epyt_flow.topology import unitscategoryid_to_str
from epyt_flow.simulation import ScenarioSimulator, ScenarioVisualizer
from epyt_flow.data.benchmarks import load_leakdb_scenarios
from epyt_flow.utils import to_seconds, plot_timeseries_data

### 1. Loading an Existing (Benchmark) Scenario

Load an existing benchmark scenario -- here we load the first Hanoi scenario of LeakDB by calling [`load_leakdb_scenarios()`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.data.benchmarks.html#epyt_flow.data.benchmarks.leakdb.load_scenarios):

In [ ]:
scenario_config, = load_leakdb_scenarios(scenarios_id=["1"], use_net1=False, verbose=True)

Load the scenario into a new instance of [`ScenarioSimulator`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator):

In [ ]:
sim = ScenarioSimulator(scenario_config=scenario_config)

Alternatively, one could load an arbitrary .inp file by setting the `f_inp` argument when creating a new [`ScenarioSimulator`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator) instance -- e.g. for loading the "Hanoi.inp" file:

```
ScenarioSimulator(f_inp="Hanoi.inp")
```

More details can be found in the [documentation](https://epyt-flow.readthedocs.io/en/stable/tut.scenarios.html).

### Working with the WDN Topology

EPyT-Flow provides easy access to the topology of the WDN, with functions for plotting the network topology, and functions for getting detailed information about the elements in the WDN.

Plot the network topology of the loaded scenario utilizing the [`ScenarioVisualizer`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_visualizer.ScenarioVisualizer):

In [ ]:
ScenarioVisualizer(sim).plot_topology()

We can get more information about the loaded WDN by calling [`get_topology()`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator.get_topology) of the simulator class:

In [ ]:
# Get network topology
topo = sim.get_topology()

The returned [`NetworkTopology`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.html#epyt_flow.topology.NetworkTopology) instance allows us to get more information:

In [ ]:
# Print all nodes and edges
print(f"Nodes: {topo.nodes}")
print(f"Edges: {topo.edges}")

Find the shortest path between nodes "2" and "22" by using the [`get_shortest_path()`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.html#epyt_flow.topology.NetworkTopology.get_shortest_path) function:

In [ ]:
print(f"Shortest path between '2' and '22': {topo.get_shortest_path('2', '22')}")

Get detailed information about a node or link by calling [`get_node_info()`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.html#epyt_flow.topology.NetworkTopology.get_node_info)/[`get_link_info()`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.html#epyt_flow.topology.NetworkTopology.get_link_info):

In [ ]:
print(f"Node '2': {topo.get_node_info('2')}")
print(f"Link '10': {topo.get_link_info('10')}")

The topology can also be converted to [`GeoDataFrame`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html) instances:

In [ ]:
# Nodes as geopandas.GeoDataFrame
topo.to_gis()["nodes"]

We can get the used units by querying the attribute [`units`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.html#epyt_flow.topology.NetworkTopology.units) and convert it to a human-readable string by calling [`unitscategoryid_to_str`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.html#epyt_flow.topology.unitscategoryid_to_str):

In [ ]:
print(unitscategoryid_to_str(topo.units))

### 3. Running a complete Scenario Simulation

Running a simulation usually involves at least the following three steps:
1. Specifying general parameters such as simulation duration and units. 
2. Specifying a sensor placement.
3. Run the simulation.

1. We can set some general parameters by calling [`set_general_parameters()`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator.set_general_parameters).
Here, we set the simulation duration to two days and the flow units to 'cubic meter per hour':

In [ ]:
sim.set_general_parameters(simulation_duration=to_seconds(days=2),
                           flow_units_id=ToolkitConstants.EN_CMH)

2. The sensor placement can be specified by manually specifying all sensors of each type.
Here, we create a couple of pressure sensors and a single flow sensor:

In [ ]:
# Place pressure sensors at nodes "13", "16", "22", and "30"
sim.set_pressure_sensors(sensor_locations=["13", "16", "22", "30"])

# Place a flow sensor at link/pipe "1"
sim.set_flow_sensors(sensor_locations=["1"])

3. We run the complete simulation by calling [`run_simulation()`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator.run_simulation). The function returns the results as an [`ScadaData`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.scada.html#epyt_flow.simulation.scada.scada_data.ScadaData) instance.

In [ ]:
scada_data = sim.run_simulation()

### 4. Post-processing the Simulation Results

Once the simulation is finished, we can access the final sensor readings and visualize the obtained time series.
More advanced post-processings such as changing the measurement units, sensor configuration, etc. are also possible -- see the [documentation](https://epyt-flow.readthedocs.io/en/stable/tut.scada.html) for details.

The easiest way to retrieve the final sensor readings is by calling the corresponding functions separately. E.g. here we retrieve the final readings at all pressure sensors by calling the [`get_data_pressures()`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.scada.html#epyt_flow.simulation.scada.scada_data.ScadaData.get_data_pressures) function which returns a [NumPy array](https://numpy.org/doc/stable/reference/generated/numpy.array.html) (first dimension is time):

In [ ]:
print(scada_data.get_data_pressures())

The retrieved sensor readings (i.e. time series) can be plotted utilizing the [`plot_timeseries_data()`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.html#epyt_flow.utils.plot_timeseries_data) function:

In [ ]:
# Plot all pressure readings over time
plot_timeseries_data(scada_data.get_data_pressures().T,
                     x_axis_label="Time (30min steps)", y_axis_label="Pressure in $m$")

In [ ]:
# Plot flow readings over time
plot_timeseries_data(scada_data.get_data_flows().T,
                     x_axis_label="Time (30min steps)", y_axis_label="Flow rate in $m^3/h$")

### Close the Simulation

Do not forget to close the simulation by calling the [`close()`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator.close) function.

In [ ]:
sim.close()

**Note:** You can avoid calling [`close()`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator.close) by opening [`ScenarioSimulator`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator) in a `with` statement -- i.e. [`close()`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator.close) is called automatically in the background:

In [ ]:
# Open 'ScenarioSimulator' in a 'with' statement to avoid calling 'close'
with ScenarioSimulator(scenario_config=scenario_config) as sim:
    # Set some general parameters
    sim.set_general_parameters(simulation_duration=to_seconds(days=2),
                               flow_units_id=ToolkitConstants.EN_CMH)
    
    # ....

    # Run simulation
    sim.run_simulation()